<a href="https://colab.research.google.com/github/ALREstevam/JupyterMapIconsBodge/blob/main/map_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install filecache
!pip install wikipedia
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=b0dafeea2e527460ee0faf2b2d8766b5b3f99feae4dc27aa5a1d573fc86ab153
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.3-py3-none-any.whl size=16463 sha256=4768d94583f216c91d37441a565b01dabf26fc210cc3a87e5e571bf4788dfa4a
  Stored in directory: /root/.cache/pip/wheels/3c/a4/19/02a1f08d032a017d5d7e22da595aa652ba0a2f2e22de73981b
Successfully built pandarallel


In [2]:
import pandas as pd
import IPython as jupyter
import folium
import json
from filecache import filecache
import wikipedia
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=8)
from jinja2 import Environment, PackageLoader, Template
import requests

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

def extract_url_title(txt):
  return txt.split("/")[-1].split(".")[0].replace("_", " ").replace("-", " ").replace("%2C", " ").replace("  ", " ").lower()

In [4]:
from wikipedia.wikipedia import summary
def wikiheader(term, lat, lon, lang):
  try:
    wikipedia.set_lang(lang)
    
    results = wikipedia.search(term)
    page = wikipedia.WikipediaPage(results[0])
    images = [im for im in page.images if (
        (
            im.endswith('.jpg') 
            or im.endswith('.jpeg')
            or im.endswith('.png')
        ) 
        and "/commons/" in im 
        and im not in [
                       'https://upload.wikimedia.org/wikipedia/commons/8/85/Clock_Tower_-_Palace_of_Westminster%2C_London_-_May_2007_icon.png'
                      ]
        )
    ]
    images = sorted(images, key=lambda test : jaccard_similarity(extract_url_title(term), extract_url_title(test)), reverse=True)
    nearby = wikipedia.geosearch(lat, lon)
    summary = page.summary

    return dict(
        images = images,
        nearby = nearby,
        summary = summary,
        similar = results,
    )
    
  except Exception as e:
    print(e)
    return {}

In [5]:
from math import radians, cos, sin, asin, sqrt

def haversine(p1, p2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    lat1, lon1 = p1
    lat2, lon2 = p2
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [6]:
def middle_point(latlon1, latlon2):
  return ((latlon1[0] + latlon2[0])/2, (latlon1[1] + latlon2[1])/2)

def dist_format(km):
  if km < 1:
    meter = km * 1000
    return "{:.0f} m".format(meter)
  return "{:.2f} km".format(km)

In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [8]:
sheet_url = 'https://docs.google.com/spreadsheets/d/13SQTdC8LQIZRFOrOfN22y0SYE581M5DiV9TG4lbgmi8'

gsheets = gc.open_by_url(sheet_url)
sheets = gsheets.worksheet('POINTS').get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df = df.dropna(axis=0)

df['LAT'] = pd.to_numeric(df['LAT'],errors='coerce')
df['LON'] = pd.to_numeric(df['LON'],errors='coerce')
points = df[["LAT", "LON", "POINT"]]
df = df.dropna(axis=0)

def apply_wiki_header(row):
  term = row['WIKI_OVERRIDE'] if row['WIKI_OVERRIDE'] != "" else row['FULL_NAME']
  return wikiheader(term, row['LAT'], row['LON'], 'en')

df["INFO"] = df.parallel_apply(apply_wiki_header, axis=1)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df)


list index out of range


In [9]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df)

In [10]:
df.head()

,POINT,ADDRESS,FULL_NAME,IS_HOME,LAT_G,LON_G,LAT,LON,COLOR,ICON,ICON_IMG,WIKI_OVERRIDE,IMGDATA,IMG,INFO
0,Notting Hill Residence by Egali,"24 Ladbroke Rd, London W11 3NN","Notting Hill Residence by Egali, 24 Ladbroke R...",TRUE,#NAME?,,51.509621,-0.201028,,https://img.icons8.com/stickers/344/bed.png,,,,,{}
1,Kensington Palace,London,"Kensington Palace, London",FALSE,#NAME?,,51.504976,-0.187681,,https://img.icons8.com/stickers/344/castle.png,,,,,{'images': ['https://upload.wikimedia.org/wiki...
2,Hyde Park,London,"Hyde Park, London",FALSE,#NAME?,,51.507268,-0.165730,,https://img.icons8.com/stickers/344/forest.png,,,,,{'images': ['https://upload.wikimedia.org/wiki...
3,Wellington Arch,London,"Wellington Arch, London",FALSE,#NAME?,,51.502592,-0.150855,,https://img.icons8.com/stickers/344/triumphal-...,,,,,{'images': ['https://upload.wikimedia.org/wiki...
4,Buckingham Palace,London,"Buckingham Palace, London",FALSE,#NAME?,,51.501364,-0.141890,,https://img.icons8.com/stickers/344/castle.png,,,,,{'images': ['https://upload.wikimedia.org/wiki...


In [11]:
def dist_near(min_dist):
  if min_dist < 1:
    return min_dist * 1.8
  if min_dist < 3: 
    return min_dist * 1.6
  if min_dist < 5: 
    return min_dist * 1.3
  if min_dist < 15:
    return min_dist * 1.2
  else:
    return min_dist * 1.05

def get_max_connections(min_dist):
  if min_dist < 1:
    return 6
  if min_dist < 3: 
    return 5
  if min_dist < 5: 
    return 4
  if min_dist < 12:
    return 
  else:
    return 2

def choose_color(r):
  if r.dist < 1:
    color = "#1cff2f"
  elif r.dist < 2:
    color = "#1c99ff"
  elif r.dist < 5:
    color = "#faee69"
  elif r.dist < 8:
    color = "#f5cb42"
  elif r.dist < 10:
    color = "#f5cb42"
  else:
    color="#ff1c2b"
  return color

In [16]:
from urllib.parse import quote




class CustomFoliumMap(folium.Map):

  _scripts = ""
  _template = Template(u"""
        {% macro header(this, kwargs) %}
            <meta name="viewport" content="width=device-width,
                initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
            <style>
                #{{ this.get_name() }} {
                    position: {{this.position}};
                    width: {{this.width[0]}}{{this.width[1]}};
                    height: {{this.height[0]}}{{this.height[1]}};
                    left: {{this.left[0]}}{{this.left[1]}};
                    top: {{this.top[0]}}{{this.top[1]}};
                }
                {{ this.get_custom_css() }}
            </style>
        {% endmacro %}

         {% macro html(this, kwargs) %}
         <div class="full reveal" id="exampleModal1" data-reveal>
         <button class="close-button" data-close aria-label="Close modal" type="button">
         <span aria-hidden="true">&times;</span>
         </button>
         <div id="modalContent"></div>
         </div>

        <div class="folium-map" id={{ this.get_name()|tojson }} ></div>
        {% endmacro %}

        {% macro script(this, kwargs) %}
            var {{ this.get_name() }} = L.map(
                {{ this.get_name()|tojson }},
                {
                    center: {{ this.location|tojson }},
                    crs: L.CRS.{{ this.crs }},
                    {%- for key, value in this.options.items() %}
                    {{ key }}: {{ value|tojson }},
                    {%- endfor %}
                }
            );

            {%- if this.control_scale %}
            L.control.scale().addTo({{ this.get_name() }});
            {%- endif %}

            {% if this.objects_to_stay_in_front %}
            function objects_in_front() {
                {%- for obj in this.objects_to_stay_in_front %}
                    {{ obj.get_name() }}.bringToFront();
                {%- endfor %}
            };
            {{ this.get_name() }}.on("overlayadd", objects_in_front);
            $(document).ready(objects_in_front);
            {%- endif %}
            
            {{ this.get_custom_script() }}

        {% endmacro %}
        """)
  

  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.default_js = [*super().default_js, 
                       ("foundation", "https://cdn.jsdelivr.net/npm/foundation-sites@6.7.5/dist/js/foundation.min.js"),
                       ]

    self.default_css = [*super().default_css, 
                 ("foundation_min", "https://cdn.jsdelivr.net/npm/foundation-sites@6.7.5/dist/css/foundation.min.css"), 
                ]

           


  def get_name(self, *args, **kwargs):
    return "map"


  def get_custom_script(self, *args, **kwargs):
    return self._scripts + """

    $(document).foundation()
    
    function setCurrentDisplayKey(key){
      localStorage.setItem('displayPlaceKey', key);
    }
    """

  def get_custom_css(self, *args, **kwargs):
    return """
    #gallery > img {
      width: auto;
      height:100vh;
      margin: 5px;
    }

    #gallery {
      display: flex;
      flex-wrap: wrap;
      justify-content: space-around;
    }
    

    """

  def add_script(self, text):
    self._scripts = f"""{self._scripts}
    {text}
    """

  


my_map = CustomFoliumMap(
    location=[points.LAT.mean(), points.LON.mean()], zoom_start=12, control_scale=True
)


def make_key(row):
  return quote(
      row['FULL_NAME'].lower().replace(" ", "_").replace('.', '').replace(',', '').replace('\'', '') + 
      f"{row['LAT']}-{row['LON']}".replace('.', '')
      )


for index, row in df.iterrows():

  endcoord = quote(f"{row['LAT']},{row['LON']}")

  data = dict(
          title=quote(row['POINT']),
          summary=row['INFO'].get('summary', ''),
          images=row['INFO'].get('images', []),
          near=[quote(el) for el in row['INFO'].get('nearby', [])],
          lat=row['LAT'],
          lon=row['LON'],
          maps=f"https://maps.google.com/?q={row['LAT']},{row['LON']}",
          citymapper=f"https://citymapper.com/directions?endcoord={endcoord}&endname={quote(row['POINT'])}"
  )

  k = make_key(row)

  my_map.add_script(
      """
      localStorage.setItem('{}', JSON.stringify({}));
      """.format(k, json.dumps(data))
  )

my_map.add_script("""

function fill(key){
    const content = JSON.parse(localStorage.getItem(key))
    var tag_id = document.getElementById('modalContent');

    const images = content.images.map(el => `
    <img src="${el}">
    `).join('')

    const near = content.near.map(el => `
    <span style="background:whitesmoke; padding: 5px; margin: 5px; border-radius: 20px;">${decodeURIComponent(el)}</span>
    `).join('')


    tag_id.innerHTML = `
<div>
  <h1>${decodeURIComponent(content.title)}</h1>
  <a href="${content.maps}" target="_blank" class="button">🌎 Google Maps</a>
  <a href="${content.citymapper}" target="_blank" class="button">🟢 Citymapper</a>
  <h2>Summary</h2>
  <section style="white-space: pre-line">${decodeURIComponent(content.summary)}</section>
  <h2>Related</h2>
  <div style='display: flex; flex-wrap: wrap; width: 100vw'>
    ${near}
  </div>
  <h2>Gallery</h2>
  <div id="gallery">
    ${images}
  </div>

</div>

`;
}
""")


my_map.add_script(
    requests.get("https://raw.githubusercontent.com/davetroy/geohash-js/master/geohash.js").text
)

my_map.add_script("""
const hash = (new URL(document.location)).searchParams.get("l")
const zoom = (new URL(document.location))?.searchParams?.get("z") ?? 15

if(hash){
	let pos = decodeGeoHash(hash)
	map.flyTo([pos.latitude[2], pos.longitude[2]], zoom)
}
""")

my_map.add_script("""
let position = undefined
map.addEventListener('moveend', (data)=>{

    const center = data.target._lastCenter ?? {lat: data.target.options.center[0], lng: data.target.options.center[1]}
    const geohash = encodeGeoHash(center.lat, center.lng)
    const zoom = data.target._zoom ?? data.target.options.zoom

    console.log({center, zoom, geohash})

    position = {center, zoom, geohash}

    var newurl = window.location.protocol + "//" + window.location.host + window.location.pathname + `?l=${geohash}&z=${zoom}`;
    window.history.pushState({path:newurl},'',newurl);

})
""")




my_map.add_script("""
var allMarkers = []
var visibleMarkers = []

map.addEventListener('moveend', (data) => {
	let markers = []

	map.eachLayer(function (layer) {
		if (layer.options && layer.options.pane === "markerPane" && layer.options.meta) {

			const inBounds = map.getBounds().contains({
				lat: layer.options.meta.lat,
				lng: layer.options.meta.lng,
			})
			markers.push({ ...layer.options.meta, inBounds })
		}

	});

	visibleMarkers = markers.filter(function (mk) { return mk.inBounds })
	allMarkers = markers

})
""")

visited = set()

my_map._name = "map"
my_map._id = ""

def gencontent(row):
  return """<h1>{}</h1>
  <button class="button" data-open="exampleModal1" onClick="fill('{}')">➕ More details</button>""".format(
      row['POINT'], 
      make_key(row)
      )


for index, row in df.iterrows():

  content = gencontent(row)

  popup = folium.Popup(folium.Html(content, script=True), max_width=1000)

  folium.Marker([row.LAT, row.LON], 
                popup=popup,
                # popup=info.POINT,
                tooltip=row.POINT,
                icon=folium.features.CustomIcon(row.ICON,icon_size=(30, 30)) if row['ICON'].startswith("http") else folium.Icon(color=row.COLOR, icon=row.ICON),
                # icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: blue">{info.POINT}</div>""")
                meta=dict(
                    markerKey=make_key(row),
                    name=str(row['POINT']),
                    iconUrl=str(row['ICON']),
                    lat=row['LAT'],
                    lng=row['LON'],
                ),
                ).add_to(my_map)


  all_distances = []
  
  for index_conn, nei in df.iterrows():
    if index == index_conn:
      continue

    all_distances.append(
        {
            "from": row.POINT,
            "to": nei.POINT,
            "from_pos": (row.LAT, row.LON), 
            "to_pos":(nei.LAT, nei.LON),
            "dist": haversine((row.LAT, row.LON),(nei.LAT, nei.LON))
        }
    )

  df_dist = pd.DataFrame.from_dict(all_distances).sort_values('dist', axis=0, ascending=True, inplace=False)
  min_dist = df_dist.dist.min()
  df_near = df_dist[df_dist.dist < dist_near(min_dist)]
  df_near = df_near[df_dist.dist < 63]
  df_near = df_near[0:get_max_connections(min_dist)]

  if f"{row.POINT}|{nei.POINT}" in visited or f"{nei.POINT}|{row.POINT}" in visited:
      continue

  visited.add(f"{row.POINT}|{nei.POINT}")
  visited.add(f"{nei.POINT}|{row.POINT}")
  
  for i, r in df_near.iterrows():
    color = choose_color(r)

    folium.PolyLine([r.from_pos, r.to_pos], color=color, weight=3, opacity=.9, stroke=True, tooltip="hello").add_to(my_map)

    folium.Marker(middle_point(r.from_pos, r.to_pos), 
                icon=folium.DivIcon(html=f"""
                <div style="font-family: sans-serif; color: white; /*background-color: rgba(70, 200, 76, 0.3);*/ font-weight: bold; white-space: nowrap; display: inline-block; -webkit-text-fill-color: white;
  -webkit-text-stroke-width: 1px;
  -webkit-text-stroke-color: black;">
                <p>{dist_format(r.dist)}</p>
                </div>""")
                ).add_to(my_map)  



my_map.save('./map.html')
my_map

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:307: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
